## План
- Строим матрицу
- Телепортация
- Нормализуем (сумма строки = 1)
- Домножаем на случайный вектор, пока не сойдется

In [31]:
import os
import urllib
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

In [32]:
filenames = os.listdir("raw/raw/")

In [33]:
len(filenames)

2381

In [34]:
topics = {}
for i, file in enumerate(filenames):
    topics[file[:-5]] = i

In [35]:
matrix = np.zeros((len(filenames), len(filenames)))

In [36]:
for file in tqdm(filenames):
    with open(f"raw/raw/{file}", encoding='utf-8') as f:
        cur_topic = file[:-5]
        bs = BeautifulSoup(f.read(), "html.parser")
        all_a = bs.find_all("a")
        for link in all_a:
            if not 'href' in link.attrs:
                continue
            href = urllib.parse.unquote(link['href'])
            if href.startswith("/wiki/") and href[6:] in topics:
                matrix[topics[cur_topic]][topics[href[6:]]] = 1

100%|██████████████████████████████████████████████████████████████████████████████| 2381/2381 [15:03<00:00,  1.92it/s]


In [19]:
for i in matrix:
    for j in range(len(i)):
        if i[j] == 0:
            i[j] = 0.001

In [20]:
row_sums = matrix.sum(axis=1)

In [21]:
norm_matrix = matrix / row_sums[:, np.newaxis]

In [22]:
vector = np.array([0.2]*len(filenames))

In [23]:
for _ in range(100):
    vector = norm_matrix.transpose().dot(vector)

In [24]:
np.take(vector, [1, 2])

array([0.90689658, 0.75579427])

In [25]:
rank = {}

In [26]:
for i, value in enumerate(vector):
    rank[i] = value

In [27]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
        
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{filenames[index][:-5]}: {value}")

In [28]:
page_rank(rank=rank)

Английский_язык: 6.716557570305046
США: 4.669699207353646
Франция: 3.889903472337673
Великобритания: 3.7580407464632963
Германия: 3.0455100931216603
Соединённые_Штаты_Америки: 2.9985954769202072
Италия: 2.4112272848694123
СССР: 2.3865402686646267
Викисклад: 2.3078883306310782
Латинский_язык: 2.3031696442651777


In [29]:
page_rank(rank=rank, word='армстронг')

Армстронг,_Луи: 0.21962529678012815
Армстронг,_Нил_Олден: 0.08217874727085496
Армстронг,_Нил: 0.07773299890096061
Армстронг,_Билли_Джо: 0.07228361513424357
Билли_Джо_Армстронг: 0.05923801519726804
Армстронг_(фамилия): 0.05483173977303546
Армстронг,_Алан: 0.054745431547958925
Армстронг,_Эдвин: 0.0487847961700028
Армстронг,_Лэнс: 0.04770257324025429
Армстронг,_Крэйг: 0.039077058692399254
